In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

from keras.datasets import imdb
from keras.preprocessing import sequence

In [25]:
max_features = 10000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train = torch.LongTensor(x_train).to('cuda')
y_train = torch.FloatTensor(y_train).to('cuda')
x_test = torch.LongTensor(x_test).to('cuda')
y_test = torch.FloatTensor(y_test).to('cuda')

train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

25000 train sequences
25000 test sequences
x_train shape: (25000, 500)
x_test shape: (25000, 500)


In [27]:
class network8(nn.Module):
    def __init__(self,):
        super().__init__()
        self.embedding = nn.Embedding(max_features, 128)
        self.lstm = nn.LSTM(128, 64, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(64, 1)

    def forward(self, x):
        # print(x.shape)
        x = self.embedding(x)
        # print(x.shape)
        _, output = self.lstm(x)
        # print(output.shape)
        # print(output[0].shape)
        output = output[0].squeeze(0)
        # print(output.shape)
        output = self.dropout(output)
        # print(output.shape)
        output = self.fc(output)
        # print(output.shape)
        return output

In [28]:
model = network8().to('cuda')
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters())

epochs = 10

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    correct = 0
    total = 0
    for i, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = model(inputs)
        # break
        loss = loss_fn(outputs.squeeze(1), labels)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        predicted = torch.round(torch.sigmoid(outputs.squeeze(1)))
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # break
    print('Epoch [%d/%d], Loss: %.4f, Train Accuracy: %.2f%%, ' % (epoch + 1, epochs, epoch_loss / len(train_loader),
                                                            100 * correct / total), end='')

    # Validation
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data in test_loader:
            inputs, labels = data
            outputs = model(inputs)
            predicted = torch.round(torch.sigmoid(outputs.squeeze(1)))
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        print('Validation Accuracy: : %f %%' % (100 * correct / total))


100%|██████████| 196/196 [00:02<00:00, 86.19it/s]


Epoch [1/10], Loss: 0.6040, Train Accuracy: 65.78%, Validation Accuracy: : 61.336000 %


100%|██████████| 196/196 [00:02<00:00, 88.22it/s]


Epoch [2/10], Loss: 0.4821, Train Accuracy: 78.02%, Validation Accuracy: : 81.240000 %


100%|██████████| 196/196 [00:02<00:00, 85.96it/s]


...

Epoch [8/10], Loss: 0.6125, Train Accuracy: 69.96%, Validation Accuracy: : 70.124000 %


100%|██████████| 196/196 [00:02<00:00, 86.72it/s]


Epoch [9/10], Loss: 0.5152, Train Accuracy: 74.23%, Validation Accuracy: : 81.196000 %


100%|██████████| 196/196 [00:02<00:00, 87.83it/s]


Epoch [10/10], Loss: 0.3603, Train Accuracy: 85.34%, Validation Accuracy: : 82.888000 %
